# Related Anime Graph
* We make an undirected graph of anime, where each anime is a vertex
* Two vertices are connected if one anime is a related to the other, where valid relations include sequels, prequels, etc.
* We then store the adjacency graph

In [ ]:
import os

import pandas as pd
from tqdm import tqdm

In [ ]:
os.chdir("../../data/processed_data")

In [ ]:
anime = pd.read_csv("anime.csv")

In [ ]:
def get_related_series(data, anime_id):
    df = pd.DataFrame({"source": [], "target": [], "relationship": []}, dtype=int)
    related = eval(data)
    if not related:
        return df

    related_series = set()
    for entry in related:
        df = pd.concat([df, pd.DataFrame.from_records([
            {
                "source": anime_id,
                "target": entry["anime_id"],
                "relationship": entry["relation"],
            }
        ])], ignore_index=True)
    return df

In [ ]:
all_related_series = []
for i in tqdm(range(len(anime))):
    anime_id = anime.iloc[i]["anime_id"]
    related_series = get_related_series(anime.iloc[i]["related_anime"], anime_id)
    all_related_series.append(related_series)
related_series_df = pd.concat(all_related_series, ignore_index=True)
related_series_df[["source", "target"]] = related_series_df[
    ["source", "target"]
].astype(int)

In [ ]:
anime_to_uid = pd.read_csv("anime_to_uid.csv")
anime_to_uid = anime_to_uid.set_index("animeid").to_dict()["uid"]
related_series_df = related_series_df.loc[
    lambda x: x.source.isin(list(anime_to_uid)) & x.target.isin(list(anime_to_uid))
]
related_series_df["source"].replace(anime_to_uid, inplace=True)
related_series_df["target"].replace(anime_to_uid, inplace=True)

In [ ]:
related_series_df.to_csv("related_series.csv", index=False)